In [599]:
import pandas as pd

In [600]:
data = pd.read_csv("data.csv", sep='\t')
prices = pd.read_csv("prices.csv", sep='\t')
quantity = pd.read_csv("quantity.csv", sep='\t', header=None)
#The quantity table has no header, I'm adding a header
quantity.columns = ['part_number', 'quantity']

In [601]:
#I'm checking if there are duplicates in data
data[data.duplicated(['part_number', 'manufacturer'])].count()

part_number     395
manufacturer    395
dtype: int64

In [602]:
#I delete duplicates and leave the first duplicate value (keep='first')
data = data.drop_duplicates(subset=['part_number','manufacturer'], keep='first')

In [603]:
#In the quantity table we have values ">10", I replace them with 11, for more convenient summation. 11 is the minimum value that satisfies the condition ">10".
quantity.loc[quantity['quantity'] == ">10", 'quantity'] = 11
#I convert the string value of the quentity column to a numeric value for the sum
quantity['quantity'] = pd.to_numeric(quantity['quantity'], errors='coerce')
#Since there are duplicate part_number values in the table, I remove duplicates and add up the values of the quantity column
quantity = quantity.groupby('part_number')['quantity'].sum().reset_index()
#I leave only those lines where quantity != 0
quantity = quantity[quantity['quantity'] != 0]
#If we need to replace quantity >10 with ">10", we can use the following
#quantity.loc[quantity['quantity'] > 10, 'quantity'] = ">10"

In [604]:
#I leave only those lines where price != 0,0
prices = prices[prices['price'] != "0,0"]

In [605]:
#I merge the tables data and prices by part_number and the part_number occurrences in both tables (how="inner")
merged_tables = pd.merge(data, prices, on="part_number", how="inner")
#I merge the tables merged_tables and quantity by part_number and the part_number occurrences in both tables (how="inner")
merged_tables = pd.merge(merged_tables, quantity, on="part_number", how="inner")

In [606]:
#I save the final result to a file "merged_tables.csv" without adding an index column
merged_tables.to_csv("merged_tables.csv", index=False)

In [610]:
#I count the number of duplicates of each value manufacturer
counts = merged_tables['manufacturer'].value_counts()
#I write the result in a txt file in the format "BRAND1 - 5000 rows
with open('number_brand.txt', 'w', encoding='utf-8') as file:
    for index, value in counts.items():
        line = f'{str(index).ljust(30)} - {str(value).rjust(30)} rows\n'
        file.write(line)